# Tutorial: Creating a Hybrid Retrieval Pipeline

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Components Used**: [`DocumentSplitter`](https://docs.haystack.deepset.ai/docs/documentsplitter), [`SentenceTransformersDocumentEmbedder`](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder), [`DocumentJoiner`](https://docs.haystack.deepset.ai/docs/documentjoiner), [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore), [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever), [`InMemoryEmbeddingRetriever`](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever), and [`TransformersSimilarityRanker`](https://docs.haystack.deepset.ai/docs/transformerssimilarityranker)
- **Prerequisites**: None
- **Goal**: After completing this tutorial, you will have learned about creating a hybrid retrieval and when it's useful.

In [1]:
import os
import sys
import warnings

import pandas as pd

from tqdm import tqdm
from uuid import uuid4

from langchain_chroma import Chroma
# from langchain_huggingface import HuggingFaceEmbeddings

# from langchain_core.documents import Document

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing

## Overview

**Hybrid Retrieval** combines keyword-based and embedding-based retrieval techniques, leveraging the strengths of both approaches. In essence, dense embeddings excel in grasping the contextual nuances of the query, while keyword-based methods excel in matching keywords.

There are many cases when a simple keyword-based approaches like BM25 performs better than a dense retrieval (for example in a specific domain like healthcare) because a dense model needs to be trained on data. For more details about Hybrid Retrieval, check out [Blog Post: Hybrid Document Retrieval](https://haystack.deepset.ai/blog/hybrid-retrieval).

## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/setting-the-log-level)

## Installing Haystack

Install Haystack and other required packages with `pip`:

In [2]:
# %%bash

# pip install haystack-ai
# pip install "datasets>=2.6.1"
# pip install "sentence-transformers>=4.1.0"
# pip install accelerate

## Initializing the DocumentStore

You'll start creating your question answering system by initializing a DocumentStore. A DocumentStore stores the Documents that your system uses to find answers to your questions. In this tutorial, you'll be using the [`InMemoryDocumentStore`](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore).

In [3]:
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

> `InMemoryDocumentStore` is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the different types of external databases that Haystack supports, see [DocumentStore Integrations](https://haystack.deepset.ai/integrations?type=Document+Store&version=2.0).

## Fetching and Processing Documents

As Documents, you will use the PubMed Abstracts. There are a lot of datasets from PubMed on Hugging Face Hub; you will use [anakin87/medrag-pubmed-chunk](https://huggingface.co/datasets/anakin87/medrag-pubmed-chunk) in this tutorial.

Then, you will create Documents from the dataset with a simple for loop.
Each data point in the PubMed dataset has 4 features:
* *pmid*
* *title*
* *content*: the abstract
* *contents*: abstract + title

For searching, you will use the *contents* feature. The other features will be stored as metadata, and you will use them to have a **pretty print** of the search results or for [metadata filtering](https://docs.haystack.deepset.ai/docs/metadata-filtering).

In [4]:
from datasets import load_dataset
from haystack import Document

# dataset = load_dataset("anakin87/medrag-pubmed-chunk", split="train")
base_path = os.path.join('/orange/ufdatastudios/dj.brinkley/predictions/data/')
financial_full_path = os.path.join(base_path, 'financial_phrase_bank/all_data-adjusted_header.csv')

dataset = load_dataset("csv", data_files=financial_full_path, split="train")

docs = []
# for doc in dataset:
#     docs.append(
#         Document(content=doc["contents"], meta={"title": doc["title"], "abstract": doc["content"], "pmid": doc["id"]})
#     )
for doc in dataset:
    docs.append(
        Document(content=doc["sentence"], meta={"sentiment": doc["sentiment"]})
    )

In [5]:
docs = docs[:33]
docs

[Document(id=63461e5f36b506ade08357e3bdb34e98c8ba387ec40e2d27f0b651cdc0416edf, content: 'According to Gran , the company has no plans to move all production to Russia , although that is whe...', meta: {'sentiment': 'neutral'}),
 Document(id=99e7dc0575e03709f270d73b2dd4f9b878b6e2b998cdc488afb27daad6fb44a2, content: 'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to hos...', meta: {'sentiment': 'neutral'}),
 Document(id=a733e18a009c0cdef1ee85366b3506eb3f869588c69b0c4d604021aa6d34a1df, content: 'The international electronic industry company Elcoteq has laid off tens of employees from its Tallin...', meta: {'sentiment': 'negative'}),
 Document(id=39e2ff2d164bed02386d1da93730042991b977cb8b62db9d4786722a19a10695, content: 'With the new production plant the company would increase its capacity to meet the expected increase ...', meta: {'sentiment': 'positive'}),
 Document(id=1f38a8a49050433e5fabd819b0bc84b4944a4f683c6797d3fe98a0feeb0c8146, content

## Indexing Documents with a Pipeline

Create a pipeline to store the data in the document store with their embedding. For this pipeline, you need a [DocumentSplitter](https://docs.haystack.deepset.ai/docs/documentsplitter) to split documents into chunks of 512 words, [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder) to create document embeddings for dense retrieval and [DocumentWriter](https://docs.haystack.deepset.ai/docs/documentwriter) to write documents to the document store.

As an embedding model, you will use [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5) on Hugging Face. Feel free to test other models on Hugging Face or use another [Embedder](https://docs.haystack.deepset.ai/docs/embedders) to switch the model provider.

> If this step takes too long for you, replace the embedding model with a smaller model such as `sentence-transformers/all-MiniLM-L6-v2` or `sentence-transformers/all-mpnet-base-v2`. Make sure that the `split_length` is updated according to your model's token limit.

In [6]:
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack import Pipeline
from haystack.utils import ComponentDevice

document_splitter = DocumentSplitter(split_by="word", split_length=512, split_overlap=32)
document_embedder = SentenceTransformersDocumentEmbedder(
    model="BAAI/bge-small-en-v1.5"
)
document_writer = DocumentWriter(document_store)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component("document_splitter", document_splitter)
indexing_pipeline.add_component("document_embedder", document_embedder)
indexing_pipeline.add_component("document_writer", document_writer)

indexing_pipeline.connect("document_splitter", "document_embedder")
indexing_pipeline.connect("document_embedder", "document_writer")

indexing_pipeline.run({"document_splitter": {"documents": docs}})

2025-10-13 16:25:03.623287: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-13 16:25:04.672771: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760387104.683240 2194741 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760387104.686446 2194741 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760387104.703197 2194741 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

{'document_writer': {'documents_written': 33}}

Documents are stored in `InMemoryDocumentStore` with their embeddings, now it's time for creating the hybrid retrieval pipeline ✅

## Creating a Pipeline for Hybrid Retrieval

Hybrid retrieval refers to the combination of multiple retrieval methods to enhance overall performance. In the context of search systems, a hybrid retrieval pipeline executes both traditional keyword-based search and dense vector search, later ranking the results with a cross-encoder model. This combination allows the search system to leverage the strengths of different approaches, providing more accurate and diverse results.

Here are the required steps for a hybrid retrieval pipeline:

### 1) Initialize Retrievers and the Embedder

Initialize a [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever) and [InMemoryBM25Retriever](https://docs.haystack.deepset.ai/docs/inmemorybm25retriever) to perform both dense and keyword-based retrieval. For dense retrieval, you also need a [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder) that computes the embedding of the search query by using the same embedding model `BAAI/bge-small-en-v1.5` that was used in the indexing pipeline:

In [7]:
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever, InMemoryEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(
    model="BAAI/bge-small-en-v1.5", device=ComponentDevice.from_str("cuda:0")
)
embedding_retriever = InMemoryEmbeddingRetriever(document_store)
bm25_retriever = InMemoryBM25Retriever(document_store)

### 2) Join Retrieval Results

Haystack offers several joining methods in [`DocumentJoiner`](https://docs.haystack.deepset.ai/docs/documentjoiner) to be used for different use cases such as `merge` and `reciprocal_rank_fusion`. In this example, you will use the default `concatenate` mode to join the documents coming from two Retrievers as the [Ranker](https://docs.haystack.deepset.ai/docs/rankers) will be the main component to rank the documents for relevancy.

In [8]:
from haystack.components.joiners import DocumentJoiner

document_joiner = DocumentJoiner()

### 3) Rank the Results

Use the [TransformersSimilarityRanker](https://docs.haystack.deepset.ai/docs/transformerssimilarityranker) that scores the relevancy of all retrieved documents for the given search query by using a cross encoder model. In this example, you will use [BAAI/bge-reranker-base](https://huggingface.co/BAAI/bge-reranker-base) model to rank the retrieved documents but you can replace this model with other cross-encoder models on Hugging Face.

In [9]:
from haystack.components.rankers import TransformersSimilarityRanker

ranker = TransformersSimilarityRanker(model="BAAI/bge-reranker-base")

TransformersSimilarityRanker is considered legacy and will no longer receive updates. It may be deprecated in a future release, with removal following after a deprecation period. Consider using SentenceTransformersSimilarityRanker instead, which provides the same functionality along with additional features.


### 4) Create the Hybrid Retrieval Pipeline

Add all initialized components to your pipeline and connect them.

In [10]:
from haystack import Pipeline

hybrid_retrieval = Pipeline()
hybrid_retrieval.add_component("text_embedder", text_embedder)
hybrid_retrieval.add_component("embedding_retriever", embedding_retriever)
hybrid_retrieval.add_component("bm25_retriever", bm25_retriever)
hybrid_retrieval.add_component("document_joiner", document_joiner)
hybrid_retrieval.add_component("ranker", ranker)

hybrid_retrieval.connect("text_embedder", "embedding_retriever")
hybrid_retrieval.connect("bm25_retriever", "document_joiner")
hybrid_retrieval.connect("embedding_retriever", "document_joiner")
hybrid_retrieval.connect("document_joiner", "ranker")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - embedding_retriever: InMemoryEmbeddingRetriever
  - bm25_retriever: InMemoryBM25Retriever
  - document_joiner: DocumentJoiner
  - ranker: TransformersSimilarityRanker
🛤️ Connections
  - text_embedder.embedding -> embedding_retriever.query_embedding (list[float])
  - embedding_retriever.documents -> document_joiner.documents (list[Document])
  - bm25_retriever.documents -> document_joiner.documents (list[Document])
  - document_joiner.documents -> ranker.documents (list[Document])

### 5) Visualize the Pipeline (Optional)

To understand how you formed a hybrid retrieval pipeline, use [draw()](https://docs.haystack.deepset.ai/docs/drawing-pipeline-graphs) method of the pipeline. If you're running this notebook on Google Colab, the generate file will be saved in "Files" section on the sidebar.

In [11]:
# hybrid_retrieval.draw("hybrid-retrieval.png")

## Testing the Hybrid Retrieval

Pass the query to `text_embedder`, `bm25_retriever` and `ranker` and run the retrieval pipeline:


In [12]:
query = "Can you identify the predictions?"

result = hybrid_retrieval.run(
    {"text_embedder": {"text": query}, "bm25_retriever": {"query": query}, "ranker": {"query": query}}
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
result

{'ranker': {'documents': [Document(id=084eec7989f0cfbbb94860b54ffffe5c0ea064f896a8dc4e4e69dfa1f4fb6eb0, content: 'FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingl...', meta: {'sentiment': 'positive', 'source_id': '2c106b768ca3455523070e94cfcdbc9e145ca808ae79f54e5d2f7c56e21ab280', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}, score: 0.00016398908337578177),
   Document(id=53f4afca60c6d97e69fe7bf80b319601dfaed593c5721ed85ee59557951fee06, content: 'Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales c...', meta: {'sentiment': 'positive', 'source_id': '72866e12a99f331230f3114fc6a6ff33f4148b762f6420f1c68318041b72962d', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': []}, score: 3.744277637451887e-05, embedding: vector of size 384),
   Document(id=c8250eb1bfc021b035d71ba172e262a21ab427b756d6a7776cbbdf3bdc56e96c, content: 'According to Gra

### Pretty Print the Results
Create a function to print a kind of *search page*.

In [14]:
def pretty_print_results(prediction):
    for doc in prediction["documents"]:
        print(doc.content, "\t", doc.score)
        # print(doc.meta["abstract"])
        print("\n", "\n")

In [15]:
pretty_print_results(result["ranker"])

FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs . 	 0.00016398908337578177

 

Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 . 	 3.744277637451887e-05

 

According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing . 	 3.7425746995722875e-05

 

The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported . 	 3.7403729947982356e-05

 

Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn . 	 3.740002284757793e-05

 

According to the

## What's next

🎉 Congratulations! You've create a hybrid retrieval pipeline!

If you'd like to use this retrieval method in a RAG pipeline, check out [Tutorial: Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline) to learn about the next steps.

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates) or [join Haystack discord community](https://discord.gg/haystack).

Thanks for reading!